In [1]:
using StanModels
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.032, 0.034, 0.033, 0.034) seconds, 0.13 seconds total
Sampling took (0.043, 0.055, 0.041, 0.056) seconds, 0.20 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -892  2.1e-02    0.95  -893  -891  -891  2.0e+03  1.0e+04  1.0e+00
accept_stat__   0.91  4.2e-03    0.11  0.67  0.95   1.0  7.4e+02  3.8e+03  1.0e+00
stepsize__      0.86  5.8e-02   0.082  0.78  0.90  0.98  2.0e+00  1.0e+01  7.6e+13
treedepth__      2.0  1.2e-01    0.74   1.0   2.0   3.0  3.9e+01  2.0e+02  1.0e+00
n_leapfrog__     4.9  9.7e-01     6.4   1.0   3.0    15  4.3e+01  2.2e+02  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00      nan      nan      nan
energy__         893  3.2e-02     1

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼─────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ mu         │ 154.607 │ 0.415175 │ 0.0065645  │ 0.00753635 │ 4000.0 │
│ 2   │ sigma      │ 7.77354 │ 0.294119 │ 0.00465043 │ 0.00562031 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%   │ 50.0%   │ 75.0%   │ 97.5%   │
│     │ Symbol     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ mu         │ 153.799 │ 154.309 │ 154.611 │ 154.894 │ 155.414 │
│ 2   │ sigma      │ 7.22077 │ 7.57183 │ 7.76719 │ 7.97136 │ 8.37996 │


end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*